In [4]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context('notebook')

import requests
from bs4 import BeautifulSoup

In [53]:
root_url = 'http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.'
report = '&ww_i_reportModel=133685247&'
filter_path = root_url + 'filter?ww_b_list=1' + report
list_path = root_url + 'html?ww_i_reportModelXsl=133685270' + report
filter_path

'http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter?ww_b_list=1&ww_i_reportModel=133685247&'

# Task 1

> Obtain all the data for the Bachelor students, starting from 2007. Keep only the students for which you have an entry for both Bachelor semestre 1 and Bachelor semestre 6. Compute how many months it took each student to go from the first to the sixth semester. Partition the data between male and female students, and compute the average -- is the difference in average statistically significant?

In [88]:
# Find the IDs for the individual years

req = requests.get(filter_path + 'ww_x_UNITE_ACAD=249847')

if req.status_code != 200:
  raise Exception('Request failed')

parser = BeautifulSoup(req.text, "html.parser")

years = parser.find('select', attrs={"name": "ww_x_PERIODE_ACAD"}).findAll('option')
year_ids = {}

for el in years:
    year_ids[el.text] = el['value']

periods = parser.find('select', attrs={'name': 'ww_x_PERIODE_PEDAGO'})
period_ids = {}

for el in periods:
    period_ids[el.text] = el['value']

period_ids

{'': 'null',
 'Bachelor semestre 1': '249108',
 'Bachelor semestre 2': '249114',
 'Bachelor semestre 3': '942155',
 'Bachelor semestre 4': '942163',
 'Bachelor semestre 5': '942120',
 'Bachelor semestre 5b': '2226768',
 'Bachelor semestre 6': '942175',
 'Bachelor semestre 6b': '2226785',
 'Master semestre 1': '2230106',
 'Master semestre 2': '942192',
 'Master semestre 3': '2230128',
 'Master semestre 4': '2230140',
 'Mineur semestre 1': '2335667',
 'Mineur semestre 2': '2335676',
 'Mise à niveau': '2063602308',
 'Projet Master automne': '249127',
 'Projet Master printemps': '3781783',
 'Semestre automne': '953159',
 'Semestre printemps': '2754553',
 'Stage automne 3ème année': '953137',
 'Stage automne 4ème année': '2226616',
 'Stage printemps 3ème année': '983606',
 'Stage printemps 4ème année': '2226626',
 'Stage printemps master': '2227132'}

In [ ]:
req = requests.get(filter_path + 'ww_x_UNITE_ACAD=249847')

In [85]:
req = requests.get(list_path + 'ww_x_GPS=71297550&ww_x_UNITE_ACAD=249847')

if req.status_code != 200:
  raise Exception('Request failed')

parser = BeautifulSoup(req.text, "html.parser")

parser.findAll('tr')[1]

<tr><th>Civilité</th><th>Nom Prénom</th><th>Orientation Bachelor</th><th>Orientation Master</th><th>Spécialisation</th><th>Filière opt.</th><th>Mineur</th><th>Statut</th><th>Type Echange</th><th>Ecole Echange</th><th>No Sciper</th></tr>

In [49]:
list(map((lambda x: list(map((lambda y: y.text), x.findAll('td')))), parser.findAll('tr')[2:]))

[['Monsieur',
  'Arévalo\xa0Christian',
  '',
  '',
  '',
  '',
  '',
  'Présent',
  '',
  '',
  '169569',
  ''],
 ['Monsieur',
  'Aubelle\xa0Flavien',
  '',
  '',
  '',
  '',
  '',
  'Présent',
  '',
  '',
  '174905',
  ''],
 ['Monsieur',
  'Badoud\xa0Morgan',
  '',
  '',
  '',
  '',
  '',
  'Présent',
  '',
  '',
  '173922',
  ''],
 ['Monsieur',
  'Baeriswyl\xa0Jonathan',
  '',
  '',
  '',
  '',
  '',
  'Présent',
  '',
  '',
  '179406',
  ''],
 ['Monsieur',
  'Barroco\xa0Michael',
  '',
  '',
  '',
  '',
  '',
  'Présent',
  '',
  '',
  '179428',
  ''],
 ['Monsieur',
  'Belfis\xa0Nicolas',
  '',
  '',
  '',
  '',
  '',
  'Présent',
  '',
  '',
  '179324',
  ''],
 ['Monsieur',
  'Bindschaedler\xa0Vincent',
  '',
  '',
  '',
  '',
  '',
  'Présent',
  '',
  '',
  '179449',
  ''],
 ['Monsieur',
  'Bloch\xa0Marc-Olivier',
  '',
  '',
  '',
  '',
  '',
  'Présent',
  '',
  '',
  '178553',
  ''],
 ['Monsieur',
  'Bloch\xa0Remi',
  '',
  '',
  '',
  '',
  '',
  'Présent',
  '',
  '',
  '17

# Task 2

> Perform a similar operation to what described above, this time for Master students. Notice that this data is more tricky, as there are many missing records in the IS-Academia database. Therefore, try to guess how much time a master student spent at EPFL by at least checking the distance in months between Master semestre 1 and Master semestre 2. If the Mineur field is not empty, the student should also appear registered in Master semestre 3. Last but not the least, don't forget to check if the student has an entry also in the Projet Master tables. Once you can handle well this data, compute the "average stay at EPFL" for master students. Now extract all the students with a Spécialisation and compute the "average stay" per each category of that attribute -- compared to the general average, can you find any specialization for which the difference in average is statistically significant?

# Bonus-Task

> BONUS: perform the gender-based study also on the Master students, as explained in 1. Use scatterplots to visually identify changes over time. Plot males and females with different colors -- can you spot different trends that match the results of your statistical tests?